In [1]:
!pip install vllm datasets
!git clone https://github.com/BatsResearch/bonito.git
!cd bonito && pip install -e .

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 kB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 26.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
!pip install "distilabel[hf-transformers, openai]>=1.0.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.0/345.0 kB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.6/241.6 kB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 106.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 46.9 MB/s eta 0:00:00
  Attempting uninstall: nest-asyncio
    Found existing installation: nest-asyncio 1.5.8
    Uninstalling nest-asyncio-1.5.8:
      Successfully uninstalled nest-asyncio-1.5.8

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -

In [29]:
from bonito import Bonito
from vllm import SamplingParams
from datasets import load_dataset, Dataset
from pydantic import Field
from distilabel.steps import Step, StepInput
from distilabel.steps.typing import StepOutput
from typing import List
from distilabel.pipeline import Pipeline
from distilabel.steps import LoadDataFromHub, KeepColumns, LoadDataFromDicts

In [44]:
class GenerateBonito(Step):

    max_tokens: int = Field(..., description="The maximum number of tokens per generation")
    top_p: float = Field(..., description="The top_p value for sampling")
    temperature: float = Field(..., description="The temperature value for sampling")
    n: int = Field(..., description="The number of synthetic examples to generate")
    context_col: str = Field(..., description="The column name of the context in the input data")
    task_type: str = Field(..., description="The type of task to generate")
    model: str = Field(..., description="The model id to use for generation")

    def __init__(self, name, **data):
        super().__init__(name=name, **data)
        self._bonito = None

    @property
    def inputs(self) -> List[str]:
        # Specify the input fields expected by this step
        return ['input']

    @property
    def outputs(self) -> List[str]:
        # Specify the output fields that this step will produce
        return ['input', 'output']

    def process(self, inputs: StepInput) -> StepOutput:
        
        if self._bonito is None:
            self._bonito = Bonito(self.model)

        sampling_params = SamplingParams(
            max_tokens=self.max_tokens, 
            top_p=self.top_p, 
            temperature=self.temperature, 
            n=self.n
        )

        input_dataset = Dataset.from_list(inputs)
        
        synthetic_dataset = self._bonito.generate_tasks(
            input_dataset,
            context_col=self.context_col,
            task_type=self.task_type,
            sampling_params=sampling_params
        )
        print(synthetic_dataset)
        yield synthetic_dataset.to_pandas().to_dict(orient='records')

In [45]:
with Pipeline(name='bonito-pipeline') as pipeline:
    load_data_from_hub = LoadDataFromHub(
        name='load-data-from-hub',
        output_mappings={"text": "input"}
    )

    synthesizer = GenerateBonito(
        name='synthesizer',
        input_batch_size = 1,
        max_tokens = 512,
        top_p = 0.5,
        temperature = 0.7,
        n = 5,
        context_col = 'input',
        task_type = 'qa',
        model = 'BatsResearch/bonito-v1',
    )
    
    load_data_from_hub >> synthesizer 


In [46]:
distiset = pipeline.run(
    parameters={
        load_data_from_hub.name: {
            'repo_id': "ahsanirfan961/title-content-dataset",
            "split": "train"
        },
    },
    use_cache=False
)

[09/14/24 06:43:42] INFO     ['distilabel.pipeline'] 📝 Pipeline data will be written to                ]8;id=324392;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=882810;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#696\696]8;;\
                             '/root/.cache/distilabel/pipelines/bonito-pipeline/290d74befb6b87510a69d97            
                             09f3b3e824cdd202f/data'                                                               

                    INFO     ['distilabel.pipeline'] ⌛ The steps of the pipeline will be loaded in     ]8;id=378739;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=621821;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#705\705]8;;\
                             stages:                                                                               
                              * Stage 0: ['load-data-from-hub', 'synthesizer']                                     

                    INFO     ['distilabel.pipeline'] ⏳ Waiting for all the steps of stage 0 to load... ]8;id=695154;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=167707;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#918\918]8;;\

[09/14/24 06:43:45] INFO     ['distilabel.pipeline'] ⏳ Steps from stage 0 loaded: 2/2                  ]8;id=655116;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=719302;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#950\950]8;;\
                              * 'load-data-from-hub' replicas: 1/1                                                 
                              * 'synthesizer' replicas: 1/1                                                        

                    INFO     ['distilabel.pipeline'] ✅ All the steps from stage 0 have been loaded!    ]8;id=869980;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=844168;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#954\954]8;;\

...

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output'],
    num_rows: 5
})


[09/14/24 06:44:47] INFO     ['distilabel.step.synthesizer'] 📨 Step 'synthesizer' sending      ]8;id=376155;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=258407;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/step_wrapper.py#274\274]8;;\
                             batch 8 to output queue                                                               

                    INFO     ['distilabel.step.synthesizer'] 🏁 Finished running step           ]8;id=458346;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=210091;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/step_wrapper.py#127\127]8;;\
                             'synthesizer' (replica ID: 0)                                                         

Generating train split: 0 examples [00:00, ? examples/s]

In [47]:
print(distiset)

Distiset({
    default: DatasetDict({
        train: Dataset({
            features: ['input', 'output'],
            num_rows: 45
        })
    })
})


In [48]:
print(distiset['default']['train'].to_pandas())

                                                input  \
0   a us navy transport plane carrying  people cra...   
1   a us navy transport plane carrying  people cra...   
2   a us navy transport plane carrying  people cra...   
3   a us navy transport plane carrying  people cra...   
4   a us navy transport plane carrying  people cra...   
5   zimbabwe s former vice president emmerson mnan...   
6   zimbabwe s former vice president emmerson mnan...   
7   zimbabwe s former vice president emmerson mnan...   
8   zimbabwe s former vice president emmerson mnan...   
9   zimbabwe s former vice president emmerson mnan...   
10  the mayor and police chief of cleveland next w...   
11  the mayor and police chief of cleveland next w...   
12  the mayor and police chief of cleveland next w...   
13  the mayor and police chief of cleveland next w...   
14  the mayor and police chief of cleveland next w...   
15  the ranking democrat on the us senate subcommi...   
16  the ranking democrat on the